## 데이터 준비
## 원본 데이터 용량이 200GB로 너무 커서 제출용으로는 샘플 파일로 제한함
## train_test_split 작업 생략하고, 파일 업로드 전 미리 train, test, validation 데이터 분리

In [ ]:
from google.colab import drive
import shutil
import zipfile

drive.mount('/content/drive')

shutil.copy('/content/drive/My Drive/data/leafdry.zip', '/content')

zip_file_train_img='/content/leafdry.zip'

with zipfile.ZipFile(zip_file_train_img, 'r') as zip_ref:
  zip_ref.extractall('/content/leafdry')

In [ ]:
#train data

## json file load and mergeing

import os
import json
import pandas as pd
import numpy as np


folder_path = r'/content/leafdry/train/train_label'
df_list = []

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)

            df_list.append(pd.json_normalize(data))
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in file {filename}: {e}")

if df_list:
    merged_df = pd.concat(df_list, ignore_index=True)
    print(merged_df)
else:
    print("No valid JSON files found.")

In [ ]:
merged_df.head(2)

In [4]:
merged_df.to_csv('leafdry_train_label_merged.csv',encoding='utf-8', index=False)

In [8]:
## preprocessing merged csv

df = pd.read_csv('leafdry_train_label_merged.csv', encoding='utf-8')

In [9]:
# 필요한 정부만 추출
  #annotations 컬럼의 레코드 최대 길이 확인 -> 최대 길이 = max bbox갯수 -> max bbox 갯수 기준으로 bbox_, name_, status_ 컬럼 생성

df_sel=df[['images.fname','images.disease_class','images.disease_cause_method','annotations','categories']].copy()

df_sel['annotations']=df_sel['annotations'].apply(lambda x: json.loads(x.replace("'","\"")))
df_sel['categories']=df_sel['categories'].apply(lambda x: json.loads(x.replace("'","\"")))

max_len=df_sel.loc[df_sel['annotations'].str.len().idxmax()]
max_len_val=max_len['annotations']
data=max_len_val

if data and isinstance(data, list):
  item=data[-1]
  cat_id=item['category_id']
  print(cat_id)
else:
  print('fuck')

length=cat_id+1
zero_box=[0,0,0,0]
unknown_status='F'

for i in range(length):
  df_sel[f'bbox_{i}'] = None
  df_sel[f'status_{i}'] = None
  df_sel[f'name_{i}'] = None

for idx, row in df_sel.iterrows():
  annotation_data=row['annotations']
  for k in range(len(annotation_data)):
    if k < len(annotation_data):
      df_sel.at[idx,f'bbox_{k}']=annotation_data[k]['bbox']
      df_sel.at[idx,f'status_{k}']=annotation_data[k]['disease_status']
    else:

      df_sel.at[idx,f'bbox_{k}']=zero_box
      df_sel.at[idx,f'status_{k}']= unknown_status

for idx, row in df_sel.iterrows():
  name_data=row['categories']
  for k in range(len(name_data)):
    if k < len(name_data):
      df_sel.at[idx,f'name_{k}']=name_data[k]['name']
    else:
      df_sel.at[idx,f'name_{k}']='unknown'


df_sel.drop(columns=['annotations','categories'], axis=1, inplace=True)

14


In [10]:
df_sel.head(2)

,images.fname,images.disease_class,images.disease_cause_method,bbox_0,status_0,name_0,bbox_1,status_1,name_1,bbox_2,...,name_11,bbox_12,status_12,name_12,bbox_13,status_13,name_13,bbox_14,status_14,name_14
0,딸기_설향_잎끝마름_88_011_221128101701.jpg,잎끝마름,칼슘부족,"[981.56, 759.33, 195.39, 162.18]",N,과실,"[1060.33, 729.79, 152.72, 72.5]",Y,과실,"[1088.79, 577.77, 105.67, 92.18]",...,화방,None,None,None,None,None,None,None,None,None
1,딸기_설향_잎끝마름_88_011_221129131701.jpg,잎끝마름,칼슘부족,"[219.96, 513.36, 352.22, 111.62]",N,줄기,"[121.22, 750.02, 369.81, 258.93]",N,잎,"[994.01, 770.45, 196.73, 144.29]",...,None,None,None,None,None,None,None,None,None,None


In [12]:
df_sel.rename(columns={'images.fname' : 'img', 'images.disease_class' : 'class', 'images.disease_cause_method' : 'method'}, inplace=True)

In [13]:
df_sel.to_csv('leafdry_train_label_preprocessed.csv',encoding='utf-8', index=False)

In [14]:
df_sel.head(1)

,img,class,method,bbox_0,status_0,name_0,bbox_1,status_1,name_1,bbox_2,...,name_11,bbox_12,status_12,name_12,bbox_13,status_13,name_13,bbox_14,status_14,name_14
0,딸기_설향_잎끝마름_88_011_221128101701.jpg,잎끝마름,칼슘부족,"[981.56, 759.33, 195.39, 162.18]",N,과실,"[1060.33, 729.79, 152.72, 72.5]",Y,과실,"[1088.79, 577.77, 105.67, 92.18]",...,화방,None,None,None,None,None,None,None,None,None


In [ ]:
#test data

In [ ]:
folder_path = r'/content/leafdry/test/label'
df_list = []

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)

            df_list.append(pd.json_normalize(data))
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in file {filename}: {e}")

if df_list:
    merged_df = pd.concat(df_list, ignore_index=True)
    print(merged_df)
else:
    print("No valid JSON files found.")

In [17]:
merged_df.to_csv('leafdry_test_label_merged.csv',encoding='utf-8', index=False)

In [18]:
df = pd.read_csv('leafdry_test_label_merged.csv', encoding='utf-8')

In [19]:
df_sel=df[['images.fname','images.disease_class','images.disease_cause_method','annotations','categories']].copy()

df_sel['annotations']=df_sel['annotations'].apply(lambda x: json.loads(x.replace("'","\"")))
df_sel['categories']=df_sel['categories'].apply(lambda x: json.loads(x.replace("'","\"")))

max_len=df_sel.loc[df_sel['annotations'].str.len().idxmax()]
max_len_val=max_len['annotations']
data=max_len_val

if data and isinstance(data, list):
  item=data[-1]
  cat_id=item['category_id']
  print(cat_id)
else:
  print('fuck')

length=cat_id+1
zero_box=[0,0,0,0]
unknown_status='F'

for i in range(length):
  df_sel[f'bbox_{i}'] = None
  df_sel[f'status_{i}'] = None
  df_sel[f'name_{i}'] = None

for idx, row in df_sel.iterrows():
  annotation_data=row['annotations']
  for k in range(len(annotation_data)):
    if k < len(annotation_data):
      df_sel.at[idx,f'bbox_{k}']=annotation_data[k]['bbox']
      df_sel.at[idx,f'status_{k}']=annotation_data[k]['disease_status']
    else:

      df_sel.at[idx,f'bbox_{k}']=zero_box
      df_sel.at[idx,f'status_{k}']= unknown_status

for idx, row in df_sel.iterrows():
  name_data=row['categories']
  for k in range(len(name_data)):
    if k < len(name_data):
      df_sel.at[idx,f'name_{k}']=name_data[k]['name']
    else:
      df_sel.at[idx,f'name_{k}']='unknown'


df_sel.drop(columns=['annotations','categories'], axis=1, inplace=True)

16


In [20]:
df_sel.rename(columns={'images.fname' : 'img', 'images.disease_class' : 'class', 'images.disease_cause_method' : 'method'}, inplace=True)
df_sel.to_csv('leafdry_test_label_preprocessed.csv',encoding='utf-8', index=False)

In [ ]:
## validation

In [ ]:
folder_path = r'/content/leafdry/validation/label'
df_list = []

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)

            df_list.append(pd.json_normalize(data))
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in file {filename}: {e}")

if df_list:
    merged_df = pd.concat(df_list, ignore_index=True)
    print(merged_df)
else:
    print("No valid JSON files found.")

In [22]:
merged_df.to_csv('leafdry_validation_label_merged.csv',encoding='utf-8', index=False)

In [23]:
df = pd.read_csv('leafdry_validation_label_merged.csv', encoding='utf-8')

In [24]:
df_sel=df[['images.fname','images.disease_class','images.disease_cause_method','annotations','categories']].copy()

df_sel['annotations']=df_sel['annotations'].apply(lambda x: json.loads(x.replace("'","\"")))
df_sel['categories']=df_sel['categories'].apply(lambda x: json.loads(x.replace("'","\"")))

max_len=df_sel.loc[df_sel['annotations'].str.len().idxmax()]
max_len_val=max_len['annotations']
data=max_len_val

if data and isinstance(data, list):
  item=data[-1]
  cat_id=item['category_id']
  print(cat_id)
else:
  print('fuck')

length=cat_id+1
zero_box=[0,0,0,0]
unknown_status='F'

for i in range(length):
  df_sel[f'bbox_{i}'] = None
  df_sel[f'status_{i}'] = None
  df_sel[f'name_{i}'] = None

for idx, row in df_sel.iterrows():
  annotation_data=row['annotations']
  for k in range(len(annotation_data)):
    if k < len(annotation_data):
      df_sel.at[idx,f'bbox_{k}']=annotation_data[k]['bbox']
      df_sel.at[idx,f'status_{k}']=annotation_data[k]['disease_status']
    else:

      df_sel.at[idx,f'bbox_{k}']=zero_box
      df_sel.at[idx,f'status_{k}']= unknown_status

for idx, row in df_sel.iterrows():
  name_data=row['categories']
  for k in range(len(name_data)):
    if k < len(name_data):
      df_sel.at[idx,f'name_{k}']=name_data[k]['name']
    else:
      df_sel.at[idx,f'name_{k}']='unknown'


df_sel.drop(columns=['annotations','categories'], axis=1, inplace=True)

22


In [25]:
df_sel.rename(columns={'images.fname' : 'img', 'images.disease_class' : 'class', 'images.disease_cause_method' : 'method'}, inplace=True)
df_sel.to_csv('leafdry_validation_label_preprocessed.csv',encoding='utf-8', index=False)

In [ ]:
# model validation data

In [ ]:
folder_path = r'/content/leafdry/model_validation/model_validation_label'
df_list = []

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)

            df_list.append(pd.json_normalize(data))
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in file {filename}: {e}")

if df_list:
    merged_df = pd.concat(df_list, ignore_index=True)
    print(merged_df)
else:
    print("No valid JSON files found.")

In [27]:
merged_df.to_csv('leafdry_model_validation_label_merged.csv',encoding='utf-8', index=False)

In [28]:
df = pd.read_csv('leafdry_model_validation_label_merged.csv', encoding='utf-8')

In [29]:
df_sel=df[['images.fname','images.disease_class','images.disease_cause_method','annotations','categories']].copy()

df_sel['annotations']=df_sel['annotations'].apply(lambda x: json.loads(x.replace("'","\"")))
df_sel['categories']=df_sel['categories'].apply(lambda x: json.loads(x.replace("'","\"")))

max_len=df_sel.loc[df_sel['annotations'].str.len().idxmax()]
max_len_val=max_len['annotations']
data=max_len_val

if data and isinstance(data, list):
  item=data[-1]
  cat_id=item['category_id']
  print(cat_id)
else:
  print('fuck')

length=cat_id+1
zero_box=[0,0,0,0]
unknown_status='F'

for i in range(length):
  df_sel[f'bbox_{i}'] = None
  df_sel[f'status_{i}'] = None
  df_sel[f'name_{i}'] = None

for idx, row in df_sel.iterrows():
  annotation_data=row['annotations']
  for k in range(len(annotation_data)):
    if k < len(annotation_data):
      df_sel.at[idx,f'bbox_{k}']=annotation_data[k]['bbox']
      df_sel.at[idx,f'status_{k}']=annotation_data[k]['disease_status']
    else:

      df_sel.at[idx,f'bbox_{k}']=zero_box
      df_sel.at[idx,f'status_{k}']= unknown_status

for idx, row in df_sel.iterrows():
  name_data=row['categories']
  for k in range(len(name_data)):
    if k < len(name_data):
      df_sel.at[idx,f'name_{k}']=name_data[k]['name']
    else:
      df_sel.at[idx,f'name_{k}']='unknown'


df_sel.drop(columns=['annotations','categories'], axis=1, inplace=True)

11


In [30]:
df_sel.rename(columns={'images.fname' : 'img', 'images.disease_class' : 'class', 'images.disease_cause_method' : 'method'}, inplace=True)
df_sel.to_csv('leafdry_model_validation_label_preprocessed.csv',encoding='utf-8', index=False)